In [1]:
import torch
import transformers

name = 'mosaicml/mpt-7b-instruct'
device = 'cuda:0'
config = transformers.AutoConfig.from_pretrained(name, trust_remote_code=True)
config.attn_config['attn_impl'] = 'triton'
config.init_device = device # For fast initialization directly on GPU!

model = transformers.AutoModelForCausalLM.from_pretrained(
  name,
  config=config,
  torch_dtype=torch.bfloat16, # Load model weights in bfloat16
  trust_remote_code=True
)


/home/ubuntu-system/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/7bf8dfd6c819cdb82e2f9d0b251f79ddd33314fb/configuration_mpt.py:114: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
/home/ubuntu-system/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/7bf8dfd6c819cdb82e2f9d0b251f79ddd33314fb/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))


pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")


tokenizer_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/457k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
from transformers import pipeline

pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=device)
diffusion_prompt = """
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
You are an excellent Stable diffusion prompt writer. Given a one-sentence story, you can write a Stable diffusion prompt.

Instructions for writing a Stable diffusion prompt with a one-sentence story:

• Use specific descriptors based on the one-sentence story details (e.g., "dense pine forest" instead of "forest").
• Detail objects mentioned in the one-sentence story (e.g., "ancient leather-bound book" instead of "book").
• Highlight emotions conveyed in the one-sentence story (e.g., "melancholic," "joyful").
• Choose a visual style, such as "35mm film," "cinematic," "watercolor", "oil painting", "3D art".
• Reference an artist or artistic style that aligns with the one-sentence story vibe.
• Do not write any additional information to the prompt. Only one single prompt covered in ".

Examples of Stable diffusion prompts:

• "Macie Grey in a Victorian dress, exploring a luminous garden with glowing plants and mystical creatures, rendered in a dreamy watercolor style with soft pastel hues."
• "Mark Hamel as an astronaut in a futuristic suit, examining an ancient monolith amidst ruins on an alien planet, captured with a 35mm film grain effect and a surreal, otherworldly color palette."
• "Baby shark with a painted face on an old wall, in the style of hyper-realistic sculptures, fragmented figures, distressed materials, tiled walls of light grays, cracked, rococo—inspired art"
• "Award-winning cinematic bioluminescent oil creature design in gold, vibrant holographic gradient blue and silver colored scheme, in the style 3d hydro — drip venom character, ray tracing reflection, prismatic lighting, realistic texture detail, vibrant electric flames coursing through oil"
• "Pope Francis wearing leather jacket is a DJ in a nightclub, mixing live on stage, giant mixing table, 4k resolution, a masterpiece"
• "Kanye West in medieval armor, standing on a cliff's edge, watching a dragon soar from misty mountains, depicted in a Renaissance painting style with dramatic chiaroscuro lighting."
• "A trench-coated Dick Tracy, silhouette against neon noir streets, searching for elusive clues of a jewel thief, visualized in a vibrant neon noir style with rain-soaked streets reflecting the city's lights."

One-sentence story: {instruction}

### Response:
"""

def predict(text):
    prompt = diffusion_prompt.format(instruction=text)

    with torch.autocast('cuda', dtype=torch.bfloat16):
        res = pipe(
            prompt,
            max_new_tokens=200,
            do_sample=True,
            use_cache=True
        )
    res = res[0]['generated_text']
    res = res[len(prompt):]
    return res

for i in range(5):
    print(f'""" Try #{i} """')
    my_text = 'Image of the fox.'
    res = predict(my_text)
    print(res)
    print('\n\n\n')

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


""" Try #0 """


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


"A fox roams through a dense pine forest, running across fallen branches and leaping from mound to mound. Beneath the shade of leaves and trunks, the fox moves confidently, glancing alertly from side to side. A lush forest scene rendered with warm pastel hues, highlighted by a soft, foggy atmosphere. From above, light filters through the trees and highlights the fox as it darts through the landscape. A painterly, dream-like feeling emphasized by the glowing colors and ethereal atmosphere, accentuated by the playful movement and expressive gestures.

Dense pine forest landscape of glowing trees and trunks, rendered in warm pastel hues and detailed with playful movement and expressive gestures, accented by a soft, ethereal atmosphere. A fox roams the foreground, highlighted from above by dappled light filtering through trees."





""" Try #1 """


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Fox in a dense pine forest rendered in a style of 35mm film grain and a surreal, otherworldly color palette.




""" Try #2 """


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


You may not know it, but the fox is a very loyal animal that loves people more than its own family! As long as you respect him for what he really is, you have his unconditional admiration.




""" Try #3 """


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


The prompt is: 
Image of the fox.
This prompt has been written in response to your one-sentence story: "A fox is a cute animal in a green forest."




""" Try #4 """
Here is a list of stable diffusion prompts:

• "The noble fox stalks the forest trail and pauses before leaping onto the low branch of a spanish oak. Rendered in a hyper-realistic watercolor style, the fox's fur is intricately detailed and appears delicate as it peers into the forest around it. The background scenery is equally rich, consisting of dense pine forest, misty morning fog, and distant hills. Small white flowers bloom along the forest floor, providing a delicate contrast to the fox's sleek brown fur. The background scene also depicts the fox's environment - forest animals and other natural elements fill the landscape, creating a vivid and realistic scene. The fox's expressive eyes and playful stance make this image a perfect example of a Stable diffusion prompt!"




